# Data migration

### Setup

In [ ]:
import pymysql
import pandas as pd
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder
from os.path import expanduser
import json
import requests
from tqdm import tqdm

In [ ]:
ENV = "prod" # or 'dev' / 'prod'

In [ ]:
sql_hostname = 'localhost'
sql_username = 'root'
sql_password = 'q1w2e3r4t5'
sql_main_database = 'wpdikdas'
sql_port = 3306
ssh_host = '10.1.9.121'
ssh_password = "vendor#1234"
ssh_user = 'vendor'
ssh_port = 2299

### SSH

In [ ]:
if ENV == "prod":
    tunnel = SSHTunnelForwarder(
            (ssh_host, ssh_port),
            ssh_username=ssh_user,
            ssh_password=ssh_password,
            remote_bind_address=(sql_hostname, sql_port))

    tunnel.start()

In [ ]:
if ENV == "prod":
    conn = pymysql.connect(
        host=sql_hostname, 
        user=sql_username,
        passwd=sql_password, 
        db=sql_main_database,
        port=tunnel.local_bind_port
    )

### Local DB

In [ ]:
if ENV == "local":
    sql_password = 'frostysnow_98'

In [ ]:
if ENV == "local":
    conn = pymysql.connect(
        host=sql_hostname, 
        user=sql_username,
        passwd=sql_password, 
        db=sql_main_database,
    )

## Get Data

In [ ]:
def get_and_store_json(query, json_filename): 
    data = pd.read_sql_query(query, conn)
    data.T.to_json(json_filename)
    return data

def get(query): 
    data = pd.read_sql_query(query, conn)
    return data

## Migrate Post

### Post: post

In [ ]:
q = '''
    SELECT p.*, a.guid AS cover_url, v.post_views_total 
    FROM wpdikdas.wpdikdas_posts p 
    JOIN wpdikdas.wpdikdas_postmeta m 
    JOIN wpdikdas.wpdikdas_posts a
    JOIN (
		SELECT post_id, sum(post_views_total) AS post_views_total 
        FROM wpdikdas.wpdikdas_post_views_realtime 
        GROUP BY post_id
	) v
    ON 
    p.ID = m.post_id AND 
    m.meta_value = a.ID AND
    p.ID = v.post_id
    
    WHERE 
    p.post_type = 'post' AND
    m.meta_key = '_thumbnail_id' AND 
    a.post_type = 'attachment'
    
    ORDER BY post_date ASC
'''

In [ ]:
post = get_and_store_json(q, "post.json")

In [ ]:
post.head()

In [ ]:
post.shape

In [ ]:
json_post = json.loads(post.T.to_json())

### Post: revision

In [ ]:
q = '''
    SELECT * FROM wpdikdas_posts
    WHERE post_type = 'revision'
'''

In [ ]:
revision = get_and_store_json(q, "post_revision.json")

In [ ]:
revision.head()

In [ ]:
json_revision = json.loads(revision.T.to_json())

## Merge both

In [ ]:
merged_post = {}
for k in json_post:
    post = json_post[k]
    merged_post[post["ID"]] = post

In [ ]:
for k in json_revision:
    rev = json_revision[k]
    
    if rev["post_parent"] in merged_post:
        post = merged_post[rev["post_parent"]]
        merged_post[rev["post_parent"]] = {**post, **rev}

## Construct legit post 

In [ ]:
from slugify import slugify
from datetime import datetime, timezone
import re

# map dikdas userid to strapi userid
userIDMapping = {
    1: 3,
    6: 4,
    7: 5,
    8: 6,
    9: 7,
}

def migrate_domain(text):
    text = re.sub('http://dikdas.kemdikbud.go.id', "{{MIGRATED_IMAGE_BASE_URL}}", text)
    text = re.sub('http://dikdasmen.kemdikbud.go.id', "{{MIGRATED_IMAGE_BASE_URL}}", text)
    
    return text

def get_time(unix_time):
    return datetime.fromtimestamp(unix_time).isoformat()

In [ ]:
def migrate_post(post):
    return {
        "titleID": post['post_title'],
        "slug": slugify(post['post_title']),
        "visitCount": post["post_views_total"],
        "created_by": {
            "id": userIDMapping[post["post_author"]]
        },
        "updated_by": {
            "id": userIDMapping[post["post_author"]]
        },
        "isApproved": True,
        "publishDate": get_time(post["post_date"]/1000),
        "coverURL": migrate_domain(post["cover_url"]),
        "body": {
            "bodyID": migrate_domain(post["post_content"])
        },
        "migrated": True,
    }

## Send Migration Request

In [ ]:
url = 'http://localhost:1337/news-articles'

In [ ]:
url = 'https://kemendikbud.herokuapp.com/news-articles'

## Becareful!

In [ ]:
for post_id in tqdm(merged_post):
    old_post = merged_post[post_id]
    new_post = migrate_post(old_post)
    requests.post(url, json=new_post)

## Post joined with its thumbnail

```
SELECT p.*, a.guid as cover_url 
FROM wpdikdas.wpdikdas_posts p 
JOIN wpdikdas.wpdikdas_postmeta m 
JOIN wpdikdas.wpdikdas_posts a
ON p.ID = m.post_id and m.meta_value = a.ID
WHERE 
p.post_type = 'post' AND
m.meta_key = '_thumbnail_id' AND 
a.post_type = 'attachment'
LIMIT 10;
```

there are 463 post attachment without post_parent out of 2925 images

```
SELECT count(*) FROM wpdikdas.wpdikdas_posts
where post_type = "attachment" and post_parent = "" and post_mime_type = "image/jpeg"
```

```
SELECT count(*) FROM wpdikdas.wpdikdas_posts
where post_mime_type = "image/jpeg"
```

## Migrate Gallery

In [ ]:
q = '''
    SELECT guid
    FROM wpdikdas.wpdikdas_posts 
    WHERE 
    post_type = 'attachment' AND
    post_mime_type = 'image/jpeg'
    ORDER BY post_date ASC
'''

In [ ]:
images = get_and_store_json(q, "images.json")

In [ ]:
images.shape

In [ ]:
images.head()

In [ ]:
images = list(images["guid"])

In [ ]:
gallery_pictures = []

temp = []
for i in range(len(images)):
    image_url = migrate_domain(images[i])
    temp.append(image_url)
    
    if (i + 1) % 50 == 0:
        gallery_pictures.append(temp)
        temp = []

## Create Gallery

In [ ]:
import random
import string

def get_random_slug(k = 8):
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=k))

In [ ]:
def migrate_gallery(gallery_id, pictures):
    return {
        "titleID": "Gallery {:02d}".format(gallery_id),
        "titleEN": "Gallery {:02d}".format(gallery_id),
        "subtitle": {
            "subtitleID": "Koleksi gambar dan foto",
            "subtitleEN": "Picture and images collection",
        },
        "slug": "gallery-{:02d}-{}".format(gallery_id, get_random_slug(8)),
        "coverURL": pictures[0],
        "pictureURLs": "\n".join(pictures),
        "created_by": {
            "id": 3,
        },
        "updated_by": {
            "id": 3,
        },
        "isActive": True,
        "migrated": True,
    }

## Send Migration Request

In [ ]:
url = 'http://localhost:1337/galleries'

In [ ]:
url = 'https://kemendikbud.herokuapp.com/galleries'

In [ ]:
c = 100

for i in tqdm(range(len(gallery_pictures))):
    new_gallery = migrate_gallery(i, gallery_pictures[i])
    requests.post(url, json=new_gallery)
    
    c -= 1
    if c == 0:
        break

## Attachments

In [ ]:
q = '''
    SELECT guid
    FROM wpdikdas.wpdikdas_posts 
    WHERE 
    post_type = 'attachment' AND
    post_mime_type != 'image/jpeg'
    ORDER BY post_date ASC
'''

In [ ]:
attachments = get_and_store_json(q, "attachments.json")

In [ ]:
attachments.shape